DS project for Segmenting and Clustering

In [3]:
import numpy as np
import pandas as pd
import json 

link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(link)[0]

table_neighbourhood = table[table['Borough']!='Not assigned'].reset_index(drop='True')
unque_neighbourhood=table_neighbourhood.groupby(['Postcode','Borough']).count().reset_index()
unque_neighbourhood['Neighborhood_new'] = 'N/A'

* the step of reprocessing of neighbourhood for groupped postcode

In [4]:
neighbourhood_dic={}
for postcode,borough, nei in zip(table_neighbourhood['Postcode'],table_neighbourhood['Borough'], table_neighbourhood['Neighbourhood']):
    new_nei = ''
    if nei == 'Not assigned':
        new_nei = borough
    else:
        new_nei = nei

    if postcode not in neighbourhood_dic.keys():
        neighbourhood_dic[postcode]= new_nei
    else:
        neighbourhood_dic[postcode] = new_nei + ', ' + neighbourhood_dic.get(postcode)
                
df_neigh = pd.DataFrame.from_dict(neighbourhood_dic, orient='index')
df_neigh.reset_index(level=0,inplace=True)
df_neigh.rename(columns={'index':'Postcode',0:'Neighborhood'},inplace=True)

result_postal_group = pd.merge(unque_neighbourhood[['Postcode','Borough']],df_neigh,on='Postcode',how='inner')
#test 
#print(result_postal_group)
#print(result_postal_group[result_postal_group['Postcode']=='M7A'])
result_postal_group.shape

(103, 3)

* import geographical coordinates data of a given postal code

In [6]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.rename(columns={'Postal Code':'Postcode'},inplace=True)
post_geo = pd.merge(result_postal_group,geo[['Postcode','Latitude','Longitude']],on='Postcode',how='inner')
post_geo

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"St. Phillips, Richview Gardens, Martin Grove G...",43.688905,-79.554724
101,M9V,Etobicoke,"Thistletown, South Steeles, Silverstone, Mount...",43.739416,-79.588437


* Clustering and creating a map 

1) filter out the data-set to deal with only data that has the word Toronto

In [16]:
post_geo_toronto = post_geo[post_geo['Borough'].str.contains('Toronto')].reset_index(drop='True')
post_geo_toronto

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, South Hill, Rathnelly, Forest...",43.686412,-79.400049


2) create a map and adding markers to map

In [14]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="toronto_trip")
location=geolocator.geocode('Toronto City, ON')
latitude=location.latitude
longitude=location.longitude
#print(location.latitude,location.longitude)

In [31]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng,borough,neighborhood in zip(post_geo_toronto['Latitude'], post_geo_toronto['Longitude'], post_geo_toronto['Borough'], post_geo_toronto['Neighborhood']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto                                                                                             

* Clustering for neighborhoods

In [27]:
kcluster_num = 5
toronto_grouped_clustering = post_geo_toronto.drop('Neighborhood',1).drop('Borough',1).drop('Postcode',1)
kmeans = KMeans(n_clusters=kcluster_num, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [28]:
post_geo_toronto.insert(0,'Cluster Lables',kmeans.labels_)

In [38]:
post_geo_toronto

,Cluster Lables,Postcode,Borough,Neighborhood,Latitude,Longitude
0,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,1,M4M,East Toronto,Studio District,43.659526,-79.340923
4,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,2,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,2,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
9,2,M4V,Central Toronto,"Summerhill West, South Hill, Rathnelly, Forest...",43.686412,-79.400049


In [33]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [36]:
import matplotlib.cm as cm
import matplotlib.colors as colors

x = np.arange(kcluster_num)
ys = [i + x + (i*x)**2 for i in range(kcluster_num)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [40]:
for lat,lng,neighborhood,cluster in zip(post_geo_toronto['Latitude'], post_geo_toronto['Longitude'], post_geo_toronto['Neighborhood'], post_geo_toronto['Cluster Lables']):
    label = folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_collor=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
    
map_clusters